In [2]:

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch.utils.data as data
import matplotlib.pyplot as plt
import torch

from utils.dataset_utils import create_rolling_window_rets_vol_array, concatenate_prices_returns
from utils.conn_data import compute_realized_ewma_vol
from loss_functions.SharpeLoss import SharpeLoss
from models.DLPO import DLPO

In [3]:
# load data
source_path = os.getcwd()
inputs_path = os.path.join(source_path, "data", "inputs")
prices = pd.read_excel(os.path.join(inputs_path, "etfs-zhang-zohren-roberts.xlsx"))

# prepare dataset
prices.set_index("date", inplace=True)
# prices = prices.shift(-1)

returns = np.log(prices).diff().dropna()
prices = prices.loc[returns.index]
features = concatenate_prices_returns(prices=prices, returns=returns)

idx = features.index
returns = returns.loc[idx].values.astype('float32')
prices = prices.loc[idx].values.astype('float32')
features = features.loc[idx].values.astype('float32')  

if returns.shape[0] == prices.shape[0] == features.shape[0]:
    pass
else:
    raise Exception("Some of the arrays have different sizes")

In [4]:
# hyperparameter
input_size = features.shape[1]
output_size = int(features.shape[1] / 2)
hidden_size = 64
num_layers = 1
learning_rate = 1e-3

seq_length = 50
train_size_perc = 0.6
train_size = int(features.shape[0] * train_size_perc)
batch_size = 10
n_epochs = 500
print_every = 10

# define arrays of rolling window observations
X, prices = create_rolling_window_rets_vol_array(return_prices=features, 
                                                 prices=prices,
                                                 seq_length=seq_length)


/Users/danieloliveira/Documents/Daniel/codes/graph-portfolio-opt/src/utils/dataset_utils.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_82vavna4yl/croot/pytorch_1686931843901/work/torch/csrc/utils/tensor_new.cpp:248.)
  return_prices_out = torch.Tensor(return_prices_out)


In [5]:
X.shape

torch.Size([4387, 50, 8])

In [9]:
X_batch, prices_batch = next(iter(train_loader))

In [10]:
X_batch.shape

torch.Size([10, 50, 8])

In [12]:
weights_pred.shape

torch.Size([10, 4])

In [11]:
# (1) model
model = DLPO(input_size=input_size,
             output_size=output_size,
             hidden_size=hidden_size,
             num_layers=num_layers,
             batch_first=True)

# (2) loss fucntions
lossfn = SharpeLoss()

# (3) optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# define train and test datasets
X_train, prices_train = X[0:train_size], prices[0:train_size]
X_test, prices_test = X[train_size:], prices[train_size:]

# define data loaders
train_loader = data.DataLoader(data.TensorDataset(X_train, prices_train), shuffle=True, batch_size=batch_size)

# (4) training procedure
training_loss_values = []
model.train()
for epoch in range(n_epochs + print_every):

    for X_batch, prices_batch in train_loader:

        optimizer.zero_grad()
        # compute forward propagation
        weights_pred = model.forward(X_batch)

        # compute loss
        loss = lossfn(prices_batch, weights_pred, ascent=True)
        
        # compute gradients and backpropagate
        loss.backward()
        optimizer.step()

    if epoch % print_every == 0:
        print("Epoch: %d, sharpe (loss): %1.5f" % (epoch, loss.item() * -1))
        training_loss_values.append(loss.item() * -1)

training_loss_df = pd.DataFrame(training_loss_values, columns=["sharpe_ratio"])

print("Average training sharpe {}".format(training_loss_df.mean().item()))

training_loss_df.hist(bins=10)
plt.show()

model.eval()

Epoch: 0, sharpe (loss): 0.40340
Epoch: 10, sharpe (loss): 0.85377


KeyboardInterrupt: 